<a href="https://colab.research.google.com/github/minlee077/CS231N_Assignments_2018/blob/master/GoogLeNet/GoogLeNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cd /content/drive/My Drive/CNN_Arch

/content/drive/My Drive/CNN_Arch


## This Note Book is made for an Implementation of GoogLeNet By Pytorch
### litte bit different with Original implementation

### - This Implementation skip sub-loss & local response normalization.
### - Datasets are CIFAR-10. so, the finally fully connected layer shape is not 1024 x 1000 but 1024 x 10
## [Original paper](https://www.cs.unc.edu/~wliu/papers/GoogLeNet.pdf)

# Package Import

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import torch.nn.functional as F

import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np

# Load and Split CIFAR-10 Dataset

In [0]:
NUM_TRAIN = 49000

# The torchvision.transforms package provides tools for preprocessing data
# and for performing data augmentation; here we set up a transform to
# preprocess the data by subtracting the mean RGB value and dividing by the
# standard deviation of each RGB value; we've hardcoded the mean and std.
transform = T.Compose([
                T.RandomResizedCrop(224),
                T.ToTensor(),
                T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
            ])

# We set up a Dataset object for each split (train / val / test); Datasets load
# training examples one at a time, so we wrap each Dataset in a DataLoader which
# iterates through the Dataset and forms minibatches. We divide the CIFAR-10
# training set into train and val sets by passing a Sampler object to the
# DataLoader telling how it should sample from the underlying Dataset.
cifar10_train = dset.CIFAR10('./datasets', train=True, download=True,
                             transform=transform)
loader_train = DataLoader(cifar10_train, batch_size=64, 
                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

cifar10_val = dset.CIFAR10('./datasets', train=True, download=True,
                           transform=transform)
loader_val = DataLoader(cifar10_val, batch_size=64, 
                        sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)))

cifar10_test = dset.CIFAR10('./datasets', train=False, download=True, 
                            transform=transform)
loader_test = DataLoader(cifar10_test, batch_size=64)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


## GPU setting

In [0]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)

using device: cuda


# Save Model Module

In [0]:
  def save(model):
    model_out_path = "./model.pth"
    torch.save(model, model_out_path)
    print("Checkpoint saved to {}".format(model_out_path))

# Flatten Moduel

In [0]:
# We need to wrap `flatten` function in a module in order to stack it
# in nn.Sequential

def flatten(x):
    N = x.shape[0] # read in N, C, H, W
    return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image
class Flatten(nn.Module):
    def forward(self, x):
        return flatten(x)
      

# Final Model Accuracy Check function

In [0]:
def check_accuracy_part34(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')   
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

# Basic Blocks For Inception Module


![Fig2](https://github.com/minlee077/CNN-Model-Implementation-Pytorch/blob/master/GoogLeNet/GoogleNet-Figure2-InceptionModule.PNG?raw=true)


In [0]:
def conv1(in_D,out_D):
    model = nn.Sequential(
        nn.Conv2d(in_D,out_D,1,1),
        nn.ReLU(),
    )
    return model

def conv1_3(in_D,mid_D,out_D):
    model = nn.Sequential(
        nn.Conv2d(in_D,mid_D,1,1),
        nn.ReLU(),
        nn.Conv2d(mid_D,out_D,3,1,1),
        nn.ReLU()
    )
    return model
    
def conv1_5(in_D,mid_D,out_D):
    model = nn.Sequential(
        nn.Conv2d(in_D,mid_D,1,1),
        nn.ReLU(),
        nn.Conv2d(mid_D,out_D,5,1,2),
        nn.ReLU()
    )
    return model
    
def max3_conv1(in_D,out_D):
    model = nn.Sequential(
        nn.MaxPool2d(3,1, 1),
        nn.Conv2d(in_D,out_D,1,1),
        nn.ReLU()
    )
    return model

# Inception Module

In [0]:
class InceptionModule(nn.Module):
  
  def __init__(self,in_D,out_D1,mid_D3,out_D3,mid_D5,out_D5,pooling):
    super(InceptionModule,self).__init__()
    
    self.conv1=conv1(in_D,out_D1)
    self.conv3=conv1_3(in_D,mid_D3,out_D3)
    self.conv5=conv1_5(in_D,mid_D5,out_D5)
    self.pool=max3_conv1(in_D,pooling)
    
  def forward(self, x):
    out1=self.conv1(x)
    out3=self.conv3(x)
    out5=self.conv5(x)
    outp=self.pool(x)
    out= torch.cat([out1,out3,out5,outp],1)
    
    return out

# Setting Network Architecture

 ![Table1](https://github.com/minlee077/CNN-Model-Implementation-Pytorch/blob/master/GoogLeNet/GoogLeNet-Table1.PNG?raw=true)

![Fig3Diagram](https://github.com/minlee077/CNN-Model-Implementation-Pytorch/blob/master/GoogLeNet/GoogleNet-Figure3-NetworkDiagram.png?raw=true)

In [0]:
class GoogLeNet(nn.Module):
  def __init__(self):
    super(GoogLeNet,self).__init__()
    
    self.prevConvlayers=nn.Sequential(
        nn.Conv2d(3,64,kernel_size=7,stride=2),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=3,stride=2,padding=1),
        nn.Conv2d(64,192,kernel_size=3,stride=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=3,stride=2),
    )
    self.Inception3layers=nn.Sequential(
        InceptionModule(192,64,96,128,16,32,32), # Inception (3a)
        InceptionModule(256,128,128,192,32,96,64), # Inception (3b)
        nn.MaxPool2d(kernel_size=3,stride=2,padding=1),

    )
    self.Inception4layers=nn.Sequential(
        InceptionModule(480, 192,96,208,16,48,64),# Inception(4a)
        InceptionModule(512,160,112,224,24,64,64),# Inception(4b)
        InceptionModule(512,128,128,256,24,64,64),# Inception(4c)
        InceptionModule(512,112,144,288,32,64,64),# Inception(4d)
        InceptionModule(528,256,160,320,32,128,128),# Inception(4e)
        nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
    )
    self.Inception5layers=nn.Sequential(
        InceptionModule(832,256,160,320,32,128,128),# Inception(5a)
        InceptionModule(832,384,192,384,48,128,128),# Inception(5b)
        
        nn.AvgPool2d(kernel_size=7,stride=1)
    )
    self.DropoutAndFClayer=nn.Sequential(
        nn.Dropout2d(p=0.4),
        Flatten(),
        nn.Linear(1024,10)
    )
    
  def forward(self, x):
    prevConv_out = self.prevConvlayers(x)
    Inception3_out = self.Inception3layers(prevConv_out)
    Inception4_out = self.Inception4layers(Inception3_out)
    Inception5_out = self.Inception5layers(Inception4_out)
    out = self.DropoutAndFClayer(Inception5_out)

    return out
    


# Train Phase
## #Part1
## Learnning rate : 1e-3
## Epoch : 15

In [0]:
model = GoogLeNet()
optimizer = optim.Adam(model.parameters(),lr=1e-3)
epochs=15

In [0]:
model = model.to(device=device)  # move the model parameters to CPU/GPU
for e in range(epochs):
    print("-------------%d / %d Epoch-------------"%(e+1,epochs))
    for t, (x, y) in enumerate(loader_train):
        model.train()  # put model to training mode
        x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
        y = y.to(device=device, dtype=torch.long)

        scores = model(x)
        loss = F.cross_entropy(scores,y)        

        # Zero out all of the gradients for the variables which the optimizer
        # will update.
        optimizer.zero_grad()

        # This is the backwards pass: compute the gradient of the loss with
        # respect to each  parameter of the model.
        loss.backward()

        # Actually update the parameters of the model using the gradients
        # computed by the backwards pass.
        optimizer.step()

        if t % print_every == 0:
            print('Iteration %d, loss = %.4f' % (t, loss.item()))
            check_accuracy_part34(loader_val, model)
            
            


-------------1 / 15 Epoch-------------
Iteration 0, loss = 2.3051
Checking accuracy on validation set
Got 119 / 1000 correct (11.90)
Iteration 100, loss = 2.2909
Checking accuracy on validation set
Got 102 / 1000 correct (10.20)
Iteration 200, loss = 2.3052
Checking accuracy on validation set
Got 119 / 1000 correct (11.90)
Iteration 300, loss = 2.2493
Checking accuracy on validation set
Got 119 / 1000 correct (11.90)
Iteration 400, loss = 2.3292
Checking accuracy on validation set
Got 199 / 1000 correct (19.90)
Iteration 500, loss = 1.9228
Checking accuracy on validation set
Got 228 / 1000 correct (22.80)
Iteration 600, loss = 1.9476
Checking accuracy on validation set
Got 174 / 1000 correct (17.40)
Iteration 700, loss = 2.1337
Checking accuracy on validation set
Got 259 / 1000 correct (25.90)
-------------2 / 15 Epoch-------------
Iteration 0, loss = 1.9676
Checking accuracy on validation set
Got 301 / 1000 correct (30.10)
Iteration 100, loss = 1.8303
Checking accuracy on validation s

In [0]:
save(model)

Checkpoint saved to ./model.pth


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type GoogLeNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type InceptionModule. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Flatten. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


## #Part2
## Learnning rate : 5e-4
## Epoch : 15

In [0]:
optimizer = optim.Adam(model.parameters(),lr=5e-4)
epochs=15

In [0]:
model = model.to(device=device)  # move the model parameters to CPU/GPU
for e in range(epochs):
    print("-------------%d / %d Epoch-------------"%(e+1,epochs))
    for t, (x, y) in enumerate(loader_train):
        model.train()  # put model to training mode
        x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
        y = y.to(device=device, dtype=torch.long)

        scores = model(x)
        loss = F.cross_entropy(scores,y)        

        # Zero out all of the gradients for the variables which the optimizer
        # will update.
        optimizer.zero_grad()

        # This is the backwards pass: compute the gradient of the loss with
        # respect to each  parameter of the model.
        loss.backward()

        # Actually update the parameters of the model using the gradients
        # computed by the backwards pass.
        optimizer.step()

        if t % print_every == 0:
            print('Iteration %d, loss = %.4f' % (t, loss.item()))
            check_accuracy_part34(loader_val, model)
            
            


-------------1 / 15 Epoch-------------
Iteration 0, loss = 1.1116
Checking accuracy on validation set
Got 631 / 1000 correct (63.10)
Iteration 100, loss = 1.0726
Checking accuracy on validation set
Got 633 / 1000 correct (63.30)
Iteration 200, loss = 1.1542
Checking accuracy on validation set
Got 619 / 1000 correct (61.90)
Iteration 300, loss = 0.9229
Checking accuracy on validation set
Got 619 / 1000 correct (61.90)
Iteration 400, loss = 0.9693
Checking accuracy on validation set
Got 626 / 1000 correct (62.60)
Iteration 500, loss = 1.1813
Checking accuracy on validation set
Got 580 / 1000 correct (58.00)
Iteration 600, loss = 0.8245
Checking accuracy on validation set
Got 620 / 1000 correct (62.00)
Iteration 700, loss = 1.0079
Checking accuracy on validation set
Got 630 / 1000 correct (63.00)
-------------2 / 15 Epoch-------------
Iteration 0, loss = 1.0587
Checking accuracy on validation set
Got 623 / 1000 correct (62.30)
Iteration 100, loss = 0.8843
Checking accuracy on validation s

In [0]:
save(model)

Checkpoint saved to ./model.pth


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type GoogLeNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type InceptionModule. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Flatten. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


## #Part3
## Learnning rate : 2.5e-4
## Epoch : 15

In [0]:
optimizer = optim.Adam(model.parameters(),lr=2.5e-4)
epochs=15

In [0]:
model = model.to(device=device)  # move the model parameters to CPU/GPU
for e in range(epochs):
    print("-------------%d / %d Epoch-------------"%(e+1,epochs))
    for t, (x, y) in enumerate(loader_train):
        model.train()  # put model to training mode
        x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
        y = y.to(device=device, dtype=torch.long)

        scores = model(x)
        loss = F.cross_entropy(scores,y)        

        # Zero out all of the gradients for the variables which the optimizer
        # will update.
        optimizer.zero_grad()

        # This is the backwards pass: compute the gradient of the loss with
        # respect to each  parameter of the model.
        loss.backward()

        # Actually update the parameters of the model using the gradients
        # computed by the backwards pass.
        optimizer.step()

        if t % print_every == 0:
            print('Iteration %d, loss = %.4f' % (t, loss.item()))
            check_accuracy_part34(loader_val, model)
            
            


-------------1 / 15 Epoch-------------
Iteration 0, loss = 0.8348
Checking accuracy on validation set
Got 696 / 1000 correct (69.60)
Iteration 100, loss = 0.8882
Checking accuracy on validation set
Got 680 / 1000 correct (68.00)
Iteration 200, loss = 0.6474
Checking accuracy on validation set
Got 680 / 1000 correct (68.00)
Iteration 300, loss = 1.0389
Checking accuracy on validation set
Got 685 / 1000 correct (68.50)
Iteration 400, loss = 0.6809
Checking accuracy on validation set
Got 681 / 1000 correct (68.10)
Iteration 500, loss = 1.0829
Checking accuracy on validation set
Got 684 / 1000 correct (68.40)
Iteration 600, loss = 0.7587
Checking accuracy on validation set
Got 693 / 1000 correct (69.30)
Iteration 700, loss = 0.7770
Checking accuracy on validation set
Got 689 / 1000 correct (68.90)
-------------2 / 15 Epoch-------------
Iteration 0, loss = 0.8605
Checking accuracy on validation set
Got 704 / 1000 correct (70.40)
Iteration 100, loss = 1.0724
Checking accuracy on validation s

In [0]:
save(model)

Checkpoint saved to ./model.pth


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type GoogLeNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type InceptionModule. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Flatten. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


# Test Phase

In [0]:
check_accuracy_part34(loader_test, model)

Checking accuracy on test set
Got 7075 / 10000 correct (70.75)
